In [ ]:
! pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle competitions download -c dphi-amsterdam-airbnb-data

airbnb_listing_validate.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
airbnb_listing_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/airbnb_listing_train.csv.zip 

Archive:  /content/airbnb_listing_train.csv.zip
replace airbnb_listing_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: airbnb_listing_train.csv  


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/airbnb_listing_train.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12901 entries, 0 to 12900
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              12901 non-null  int64  
 1   name                            12879 non-null  object 
 2   host_id                         12901 non-null  int64  
 3   host_name                       12893 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   12901 non-null  object 
 6   latitude                        12901 non-null  float64
 7   longitude                       12901 non-null  float64
 8   room_type                       12901 non-null  object 
 9   minimum_nights                  12901 non-null  int64  
 10  number_of_reviews               12901 non-null  int64  
 11  last_review                     11305 non-null  object 
 12  reviews_per_month               

In [ ]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price
0,11602914,Luxury home 100m2+garden in center!,3123809,Marjolein,nan,De Pijp - Rivierenbuurt,52.35,4.90,Entire home/apt,3,13,15-02-2020,0.27,1,0,220
1,13289321,East is where your home is!,10259430,Ana,nan,Oud-Oost,52.36,4.92,Entire home/apt,4,14,19-06-2019,0.29,1,0,110
2,40779315,Amsterdam Central - LUXURY house in BUSY STREET,224969266,Mark,nan,Centrum-West,52.38,4.90,Entire home/apt,2,9,16-03-2020,1.65,1,7,100
3,7820311,great appartment down town,693472,Ayden,nan,Westerpark,52.38,4.87,Entire home/apt,3,42,17-02-2020,0.72,1,0,130
4,27346603,Private room with canal view in peaceful area,41888346,Alissa,nan,Westerpark,52.38,4.87,Private room,2,89,26-02-2020,4.02,1,24,90


In [ ]:
df.isnull().sum()

id                                    0
name                                 22
host_id                               0
host_name                             8
neighbourhood_group               12901
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
minimum_nights                        0
number_of_reviews                     0
last_review                        1596
reviews_per_month                  1596
calculated_host_listings_count        0
availability_365                      0
price                                 0
dtype: int64

In [ ]:
df[df['reviews_per_month'].isnull()==True]

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price
6,1955400,Gorgeous family home near Museum Sq,1326792,Edward,nan,Centrum-Oost,52.36,4.90,Entire home/apt,2,0,NaN,nan,1,0,225
13,16565978,Historic Dutch Barge - 1870s Built - Sleeps 4,1446243,Alasdair,nan,Centrum-Oost,52.37,4.92,Entire home/apt,21,0,NaN,nan,1,0,20
21,18584642,Amsterdam,129037804,Jolanda,nan,De Aker - Nieuw Sloten,52.34,4.79,Entire home/apt,4,0,NaN,nan,1,0,250
23,41515131,Super Unique Stay At Authentic Ship Amsterdam,140199696,Douwe,nan,Oostelijk Havengebied - Indische Buurt,52.38,4.94,Private room,2,0,NaN,nan,2,70,72
34,38538410,Gorgeous private room with private bathroom,272936189,Laura,nan,Bos en Lommer,52.38,4.86,Private room,1,0,NaN,nan,1,0,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12870,29003215,Tower Studio at Yays Oostenburgergracht,218582446,Yays Oostenburgergracht,nan,Centrum-Oost,52.37,4.92,Hotel room,1,0,NaN,nan,6,358,191
12874,13168032,Appartment,14321898,Georg,nan,De Pijp - Rivierenbuurt,52.35,4.90,Entire home/apt,3,0,NaN,nan,1,0,240
12876,37870419,Unique design apartment in centre of Amsterdam,8843719,Valérie,nan,Centrum-West,52.37,4.88,Entire home/apt,2,0,NaN,nan,1,15,300
12885,41910871,"Hotel Estheréa, Deluxe Room",316681026,Hotel Estheréa,nan,Centrum-West,52.37,4.89,Private room,1,0,NaN,nan,5,364,7000


In [ ]:
mean = df['reviews_per_month'].mean()
df['reviews_per_month'].fillna(mean,inplace = True)

In [ ]:
df.drop(columns = ['name','host_name','last_review','neighbourhood_group'],inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12901 entries, 0 to 12900
Data columns (total 12 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              12901 non-null  int64  
 1   host_id                         12901 non-null  int64  
 2   neighbourhood                   12901 non-null  object 
 3   latitude                        12901 non-null  float64
 4   longitude                       12901 non-null  float64
 5   room_type                       12901 non-null  object 
 6   minimum_nights                  12901 non-null  int64  
 7   number_of_reviews               12901 non-null  int64  
 8   reviews_per_month               12901 non-null  float64
 9   calculated_host_listings_count  12901 non-null  int64  
 10  availability_365                12901 non-null  int64  
 11  price                           12901 non-null  int64  
dtypes: float64(3), int64(7), object(

In [ ]:
df = pd.get_dummies(df,columns = ['room_type'])

In [ ]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
df['neighbourhood'] = lb.fit_transform(df['neighbourhood'])

In [ ]:
df['host_id'] = df['host_id']/max(df['host_id'])
# df['id'] = df['id']/max(df['id'])

In [ ]:
X = df.drop(columns = ['price'])
y = df.price

print(X.shape,y.shape)

(12901, 14) (12901,)


## **Training**


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import metrics

Splitting data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state =  42)
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(10320, 14) (2581, 14) (10320,) (2581,)


LinearRegression

In [ ]:
model = linear_model.LinearRegression()
model.fit(x_train,y_train)
pred = model.predict(x_test)
print((metrics.mean_squared_error(y_test,pred))**0.5)

218.07484136895755


RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(x_train,y_train)
pred = model.predict(x_test)
print((metrics.mean_squared_error(y_test,pred))**0.5)

203.46809611887804


ExtraTreesRegressor

In [ ]:
# from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(x_train,y_train)
pred = model.predict(x_test)
print((metrics.mean_squared_error(y_test,pred))**0.5)

182.520671480113


In [ ]:
test_new = pd.read_csv('/content/airbnb_listing_validate.csv')

In [ ]:
test_new.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,12297872,"Great apartment, super location",44144487,Azarakhsh,nan,De Baarsjes - Oud-West,52.37,4.86,Entire home/apt,2,49,15-09-2019,0.99,1,267
1,3920205,Lovely apartment in Center south!,7594884,"Ruben, Sarah & Lindy",nan,De Pijp - Rivierenbuurt,52.35,4.89,Entire home/apt,3,34,19-09-2019,0.49,38,0
2,26614345,"Sunny family friendly house, centrally located",3903937,Maaike,nan,Oud-Noord,52.39,4.91,Entire home/apt,4,3,21-04-2019,0.13,1,0
3,7433172,1 Br Apartment in Old South A'dam,24969223,Quirien,nan,Zuid,52.34,4.86,Entire home/apt,4,0,NaN,nan,1,0
4,17019744,Luxurious house with garden - central location,114138837,Sietse,nan,Westerpark,52.37,4.87,Entire home/apt,2,9,24-09-2017,0.23,1,0


In [ ]:
test_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6451 entries, 0 to 6450
Data columns (total 15 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              6451 non-null   int64  
 1   name                            6440 non-null   object 
 2   host_id                         6451 non-null   int64  
 3   host_name                       6448 non-null   object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   6451 non-null   object 
 6   latitude                        6451 non-null   float64
 7   longitude                       6451 non-null   float64
 8   room_type                       6451 non-null   object 
 9   minimum_nights                  6451 non-null   int64  
 10  number_of_reviews               6451 non-null   int64  
 11  last_review                     5655 non-null   object 
 12  reviews_per_month               56

In [ ]:
mean = test_new['reviews_per_month'].mean()
test_new['reviews_per_month'].fillna(mean,inplace = True)
test_new.drop(columns = ['name','host_name','last_review','neighbourhood_group'],inplace = True)

In [ ]:
test_new = pd.get_dummies(test_new,columns = ['room_type'])
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
test_new['neighbourhood'] = lb.fit_transform(test_new['neighbourhood'])

In [ ]:
test_new['host_id'] = test_new['host_id']/max(test_new['host_id'])
ids = test_new['id']
# test_new['id'] = test_new['id']/max(test_new['id'])

In [ ]:
prediction = model.predict(test_new)
predict = []
for i in prediction:
  predict.append(int(i))

In [ ]:
res = pd.DataFrame(predict)
res.index = ids 
res.columns = ["price"]
# To download the csv file locally
from google.colab import files
res.to_csv('prediction_results.csv')         

In [ ]:
!kaggle competitions submit -c dphi-amsterdam-airbnb-data -f '/content/prediction_results.csv' -m "Message"

100% 79.4k/79.4k [00:07<00:00, 10.7kB/s]
403 - Your team has used its submission allowance (20 of 20). This resets at midnight UTC (6.3 hours from now).
